In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Building Retrieval Models easily with Merlin Models

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output. Such models can be efficiently served by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine and during inference scoring user embeddings over all indexed item embeddings within the engine.In this notebook we will build a Two-Tower architecture for item retrieval.



### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using the Ali-CCP: Alibaba Click and Conversion Prediction dataset. To download the training and test datasets visit Ali-CCP: Alibaba Click and Conversion Prediction at [tianchi.aliyun.com](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1). We have curated the raw dataset via using this [script]() and generated the parquet files that we will use for this example.

### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Perform prediction for a given user query

### Feature Engineering with NVTabular

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-22 19:32:04.557591: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 19:32:05.948990: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-22 19:32:05.949120: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


First, we define our input and output paths.

In [3]:
train_path = '/workspace/data/train/*.parquet'
test_path = '/workspace/data/test/*.parquet'
output_path = '/workspace/retrieval/processed/'

ETL Workflow

We select only positive interaction rows therefore we remove rows where `click==0` from the dataset with `Filter()` op.

In [4]:
user_id = ["user_id"] >> Categorify() >> AddMetadata(tags=[Tags.USER_ID, Tags.USER])
item_id = ["item_id"] >> Categorify() >> AddMetadata(tags=[Tags.ITEM_ID, Tags.ITEM]) 

item_features = ["item_category", "item_shop", "item_brand"] \
     >> AddMetadata(tags=[Tags.ITEM]) >> nvt.ops.Categorify()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> AddMetadata(tags=[Tags.USER]) >> nvt.ops.Categorify()

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow_fit_transform(outputs, train_path, test_path, output_path)

## Building Two-Tower Model

We will use Two-Tower Model for Item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. It is Thus, these systems are often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). You can read more about two stage Recommender Systems [here](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf). In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [5]:
schema = TensorflowMetadata.from_proto_text_file('/workspace/retrieval/processed/train/').to_merlin_schema()

In [6]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We expect the label names to be empty.

In [ ]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

### Negative sampling
Many datasets for recommender systems contains implicit feedback, with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [7]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [8]:
batch_size = 4096

train = Dataset(os.path.join(output_path + '/train/*.parquet'), part_size="500MB")
valid = Dataset(os.path.join(output_path + '/test/*.parquet'), part_size="500MB")

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [9]:
model.set_retrieval_candidates_for_evaluation(train)

opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(optimizer=opt, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=batch_size, epochs=5)

2022-03-22 19:32:07.059318: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


384/384 [==============================] - ETA: 0s - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3208 - regularization_loss: 0.0000e+00 - total_loss: 8.3208

INFO:tensorflow:Assets written to: /tmp/tmpz_ytut1y/assets


...

INFO:tensorflow:Assets written to: /tmp/tmp87k6tcun/assets


384/384 [==============================] - 29s 76ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3176 - regularization_loss: 0.0000e+00 - total_loss: 8.3176 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0011 - val_loss: 7.4173 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4173
Epoch 5/5
383/384 [============================>.] - ETA: 0s - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3201 - regularization_loss: 0.0000e+00 - total_loss: 8.3201

INFO:tensorflow:Assets written to: /tmp/tmpsqngcedk/assets


INFO:tensorflow:Assets written to: /tmp/tmpsqngcedk/assets


384/384 [==============================] - 30s 78ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3171 - regularization_loss: 0.0000e+00 - total_loss: 8.3171 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0011 - val_loss: 7.4170 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4170


In batch sampling is prone to popularity-bias. since different users might have interacted same items. # in the fit method we are only using the negative sampling in each batch and the final score is avg over all the batches. 
for each batch we aer comparing only the items in the given batch. in the `.evaluate()` we are considering entire item catalog for each positive item. generate the scores for all items and check the position of the positive item in the list of scores.

In [22]:
#model.evaluate(valid, return_dict=True, batch_size=1024)

## Exporting Retrieval Models

#### Save user (query) tower

In [11]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

INFO:tensorflow:Assets written to: query_tower/assets


INFO:tensorflow:Assets written to: query_tower/assets


#### Extract and save user features

In [16]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [17]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
1970,0,0,1,4,1,3,2,1,0,0,0,0
315873,1,109,0,0,0,0,0,0,0,69,131,9
203115,2,301,1,1,1,1,1,1,2,57,4709,57
696854,3,1876,23,7,2,3,1,1,1,5,63,3
213309,4,534,1,2,1,2,1,1,0,40,22,108


In [14]:
user_features.shape

(214994, 12)

In [16]:
user_features.to_parquet('user_features.parquet')

#### Extract and save item features

In [23]:
# train path should be the path for the processed parquet files.
item_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [24]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,0,0,0,0
1,1,140,380,6
2,2,1,4,33
3,3,38,57,0
4,4,11,65,138


In [16]:
#item_features.to_parquet('item_features.parquet')

#### Extract and save item embeddings

In [26]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

INFO:tensorflow:Assets written to: /tmp/tmpuyckfegj/assets


INFO:tensorflow:Assets written to: /tmp/tmpuyckfegj/assets


In [27]:
item_embs_df

,item_id,item_category,item_shop,item_brand,0,1,2,3,4,5,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,-0.119417,-0.027896,-0.264912,-0.050182,-0.138162,0.136211,...,0.158456,0.168403,-0.096095,-0.021504,-0.123101,0.128763,0.225422,-0.106285,-0.055848,0.061402
1,1,140,380,6,-0.294812,-0.008678,0.051245,0.017450,-0.060454,0.161752,...,0.306396,-0.135012,0.022356,0.004448,-0.145180,0.078878,0.081469,-0.087589,-0.113694,-0.059155
2,2,1,4,33,-0.245789,0.050664,0.149792,0.196821,-0.051188,0.003327,...,0.161846,-0.091704,-0.039100,-0.145625,-0.172513,-0.071293,-0.054982,0.037903,0.058987,-0.066160
3,3,38,57,0,-0.049673,-0.084216,0.141419,0.067517,-0.060590,0.170737,...,0.276075,-0.128361,-0.251821,-0.107160,-0.190219,-0.062546,-0.087450,-0.105357,-0.001375,-0.012447
4,4,11,65,138,-0.192909,0.012221,-0.054791,0.124527,-0.043193,0.071654,...,0.122809,-0.177446,-0.115998,0.020514,-0.078979,-0.144547,-0.003251,-0.113254,-0.025953,0.129219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568842,568842,390,213734,77789,-0.255753,-0.087150,-0.060789,0.058536,0.070155,0.158219,...,0.142069,-0.109075,-0.136190,-0.186605,-0.085247,-0.076334,0.005534,-0.066224,0.081726,0.011914
568843,568843,137,111,161,-0.314179,-0.042740,-0.048638,0.096127,-0.092761,0.216961,...,0.137687,-0.045866,-0.066166,0.099646,-0.128875,0.024852,-0.050241,-0.081949,-0.098082,0.123643
568844,568844,543,140774,56005,-0.097131,-0.001878,-0.084501,0.101683,-0.216845,-0.027871,...,0.108308,-0.185961,-0.104254,-0.100263,-0.199770,0.112065,0.056785,-0.242739,-0.034991,-0.240517
568845,568845,803,142760,17371,-0.268720,-0.156125,-0.052229,0.026334,-0.115524,-0.216717,...,-0.000883,-0.166568,0.038430,-0.166113,-0.057967,-0.178316,-0.238665,-0.053779,-0.163326,-0.017178


In [28]:
item_embeddings = item_embs_df.iloc[:, 4:]

In [29]:
item_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.119417,-0.027896,-0.264912,-0.050182,-0.138162,0.136211,-0.169776,-0.254903,-0.154044,-0.307413,...,0.158456,0.168403,-0.096095,-0.021504,-0.123101,0.128763,0.225422,-0.106285,-0.055848,0.061402
1,-0.294812,-0.008678,0.051245,0.017450,-0.060454,0.161752,-0.037283,0.018863,-0.275601,-0.062120,...,0.306396,-0.135012,0.022356,0.004448,-0.145180,0.078878,0.081469,-0.087589,-0.113694,-0.059155
2,-0.245789,0.050664,0.149792,0.196821,-0.051188,0.003327,-0.058045,-0.021776,-0.006233,-0.129631,...,0.161846,-0.091704,-0.039100,-0.145625,-0.172513,-0.071293,-0.054982,0.037903,0.058987,-0.066160
3,-0.049673,-0.084216,0.141419,0.067517,-0.060590,0.170737,-0.031970,0.087300,-0.148692,-0.155614,...,0.276075,-0.128361,-0.251821,-0.107160,-0.190219,-0.062546,-0.087450,-0.105357,-0.001375,-0.012447
4,-0.192909,0.012221,-0.054791,0.124527,-0.043193,0.071654,-0.098000,-0.130967,-0.186607,0.031632,...,0.122809,-0.177446,-0.115998,0.020514,-0.078979,-0.144547,-0.003251,-0.113254,-0.025953,0.129219


In [25]:
item_embeddings.to_parquet('item_embeddings')